In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key=os.getenv("GEMINI_API_KEY")

In [3]:
model_client=OpenAIChatCompletionClient(
  model="gemini-1.5-flash-8b",
  api_key=api_key
)

In [4]:
agent=AssistantAgent(
  name="Writer",
  system_message="You are a creative story writer",
  model_client=model_client
)

In [5]:
async def main():
  task=TextMessage(
    content="Write a short story on Titanic",
    source='user'
  ),
  response=await agent.run(task=task)
  print(response.messages[-1].content)


await main()



The salt spray stung Elara's face as she clung to the railing, the rhythmic creak of the Titanic a lullaby against the churning grey sea.  She wasn't a passenger of the grandest class, nor a hopeful immigrant. Elara was a stowaway, a whisper of a girl hidden amongst crates of china and sacks of coal.  Her name, whispered in the dark corners of the ship, was simply Elara.

She'd known the thrill of the open sea since she was a child, her small fishing village swallowed by the endless horizon.  But this… this was different.  This was a gilded cage, a monument to human ambition and folly, all gleaming white and steel. She watched the opulent decks, the elegant gatherings, her heart a knot of longing and fear.  She wished she could see herself reflected in the polished windows of the First Class dining room, but the only reflection she found was in the depths of the endless, starless ocean.

Days bled into weeks, each a blur of shadows and whispers.  Elara learned to navigate the labyrinth

## Multi Agent Approach

In [6]:
plot_agent=AssistantAgent(
  name="plot_writer",
  model_client=model_client,
  system_message="You are a creative plot writer for storries"
)

character_agent=AssistantAgent(
  name="character_writer",
  model_client=model_client,
  system_message="You are a creative writer for stories write character writing for the story"
)

conclusion_agent=AssistantAgent(
  name="story_conclude_writer",
  model_client=model_client,
  system_message="You are a creative story writer for writing conclusion"
)

In [9]:
from autogen_agentchat.teams import RoundRobinGroupChat

multi_agent=RoundRobinGroupChat(
  [plot_agent,character_agent,conclusion_agent],
  max_turns=3
)

async def multiagent():
  task=TextMessage(
    content="Write a story on interstellar movie in 500 words",
    source='user'
  )
  response=await multi_agent.run(task=task)

  for message in response.messages:
    print(f"\033[1m{message.source}\033[0m : {message.content}")


await multiagent()

user : Write a story on interstellar movie in 500 words
plot_writer : The crimson nebula, a swirling vortex of dying stars, was the backdrop for the *Stardust Wanderer's* desperate flight.  Captain Eva Rostova, her face etched with fatigue, stared at the kaleidoscopic display.  This wasn't a mission of exploration; it was a race against time.

A distress signal, faint and fragmented, had echoed through the void, originating from a system on the edge of known space, a system consumed by a phenomenon dubbed "The Whisper".  It warped space, shattered light, and whispered promises of oblivion to any who ventured near.

Eva, a veteran pilot with a reputation for calculated risks, knew the *Stardust Wanderer* was their last hope for Dr. Aris Thorne, a brilliant astrophysicist and the sole survivor of a research team vanished into the Whisper's embrace.

Her co-pilot, Jax, a young, fiercely logical engineer, adjusted the ship's course. "Captain, the Whisper's distortions are intensifying.  We

In [22]:
async for messsage in multi_agent.run_stream(task="Write a story on titanic"):
  print(messsage)                


id='c096643c-323b-495e-a6a6-701e020e63fc' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 28, 7, 44, 40, 604081, tzinfo=datetime.timezone.utc) content='Write a story on titanic' type='TextMessage'
id='5fe8c954-c89a-4660-9e58-8b3fbd9b94a3' source='plot_writer' models_usage=RequestUsage(prompt_tokens=22409, completion_tokens=596) metadata={} created_at=datetime.datetime(2025, 7, 28, 7, 44, 46, 110129, tzinfo=datetime.timezone.utc) content='The icy wind, a mournful dirge, howled across the deck of the *Titanic*.  Eleanor Vance, a young woman whose eyes held the stormy grey of the encroaching night, clutched her furs tighter.  This magnificent vessel, a symbol of human ambition, felt strangely fragile beneath her feet, a delicate blossom threatened by the relentless ocean.  A premonition, sharp and cold as the wind itself, settled deep within her.\n\nDeep within the *Titanic\'s* iron heart, in the engine room\'s relentless symphony of clanking metal and hi

In [24]:
plot_agent=AssistantAgent(
  name="plot_writer",
  model_client=model_client,
  system_message="You are a creative plot writer for storries in 10 words"
)

character_agent=AssistantAgent(
  name="character_writer",
  model_client=model_client,
  system_message="You are a creative writer for stories write character writing for the story in 10 words"
)

conclusion_agent=AssistantAgent(
  name="story_conclude_writer",
  model_client=model_client,
  system_message="You are a creative story writer for writing conclusion in 10 words"
)

In [25]:
from autogen_agentchat.teams import RoundRobinGroupChat

multi_agent=RoundRobinGroupChat(
  [plot_agent,character_agent,conclusion_agent],
  max_turns=3
)

In [26]:
from autogen_agentchat.base import TaskResult

async for message in multi_agent.run_stream(task="Write a story on pollution"):
  if isinstance(message,TaskResult):
    print(f"Stop Reason :{message.stop_reason}")
  else:
    print(message)

id='faf03f93-c978-4da4-9f08-642fa357eb01' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 28, 7, 50, 29, 887260, tzinfo=datetime.timezone.utc) content='Write a story on pollution' type='TextMessage'
id='87fe070e-83d2-43da-aa1d-4d7567e252d5' source='plot_writer' models_usage=RequestUsage(prompt_tokens=19, completion_tokens=17) metadata={} created_at=datetime.datetime(2025, 7, 28, 7, 50, 32, 274873, tzinfo=datetime.timezone.utc) content='A city choked by smog, a lone child discovers a hidden, vibrant oasis.\n' type='TextMessage'
id='993d8b39-6adb-49c7-8204-70283cef9b9b' source='character_writer' models_usage=RequestUsage(prompt_tokens=40, completion_tokens=246) metadata={} created_at=datetime.datetime(2025, 7, 28, 7, 50, 34, 58209, tzinfo=datetime.timezone.utc) content="The air hung thick and grey, a concrete blanket smothering the city of Ashfall.  Ten-year-old Elara, eyes stinging from the perpetual haze, stumbled upon a hidden courtyard tucked behind 

In [27]:
from autogen_agentchat.ui import Console

await Console(multi_agent.run_stream(task="Write a story on pollution"))

---------- TextMessage (user) ----------
Write a story on pollution
---------- TextMessage (plot_writer) ----------
A forgotten river, choked by plastic, whispered secrets of a dying world to a boy who found it.

---------- TextMessage (character_writer) ----------
The rusted metal grating groaned under Kai's weight, revealing a chasm choked with a grotesque tapestry of plastic.  Not trash, but a tangible, suffocating blanket – bottles, bags, and wrappers, woven together by the relentless current of the forgotten river.  It snaked through the city's underbelly, a once-vital artery now a festering wound.  Kai, scavenging for scrap metal, stumbled upon it, drawn by the whispers of wind sighing through the tangled debris.

The air hung thick and fetid, the river itself a murky, oily sheen.  He knelt, tracing the contours of a shattered soda bottle, the faded logo a stark reminder of a world he scarcely knew. He saw the reflections of a grey sky in the murky water, a sky he'd only known fr

TaskResult(messages=[TextMessage(id='857b0df1-51b5-4cd1-bd32-704095075571', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 28, 7, 52, 38, 304725, tzinfo=datetime.timezone.utc), content='Write a story on pollution', type='TextMessage'), TextMessage(id='53df798f-eac6-4917-8263-fa5064af3cb3', source='plot_writer', models_usage=RequestUsage(prompt_tokens=413, completion_tokens=22), metadata={}, created_at=datetime.datetime(2025, 7, 28, 7, 52, 40, 314588, tzinfo=datetime.timezone.utc), content='A forgotten river, choked by plastic, whispered secrets of a dying world to a boy who found it.\n', type='TextMessage'), TextMessage(id='9f5a6ccd-72bb-4a2e-9c39-e93d47149cf1', source='character_writer', models_usage=RequestUsage(prompt_tokens=439, completion_tokens=420), metadata={}, created_at=datetime.datetime(2025, 7, 28, 7, 52, 42, 897475, tzinfo=datetime.timezone.utc), content="The rusted metal grating groaned under Kai's weight, revealing a chasm choked wit